In [1]:
#Importing all required Modules

from sqlalchemy.engine import create_engine
import pandas as pd
import datetime
from sqlalchemy.dialects.oracle import DATE, VARCHAR2, FLOAT, NUMBER,TIMESTAMP
from sqlalchemy import text,String,types
from datetime import datetime

In [2]:
# MySQL connection configuration

#1 : Connecting to Source:

mysql_host = 'database-1.cq5kggbpkyu9.us-east-2.rds.amazonaws.com'
mysql_user = 'aroha_analytics'
mysql_password = 'pEjw_41fA6TpB2'
mysql_database = 'AceExtraction'
source_path = f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}'

source_engine = create_engine(source_path)

#2 : Connecting to Staging DB :

mysql_host = 'arohaanalytics-stg.cq5kggbpkyu9.us-east-2.rds.amazonaws.com'
mysql_user = 'aroha_manager'
mysql_password = 'k30N053X!lb0kD'
mysql_database = 'staging_source'
staging_path = f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}'

staging_engine = create_engine(staging_path)

#3 : Connecting to DataWarehouse

mysql_host = 'arohaanalytics-datawarehouse.cq5kggbpkyu9.us-east-2.rds.amazonaws.com'
mysql_user = 'aroha_manager'
mysql_password = r'mK61\zaUM8i861'
mysql_database = 'datawarehouse_fastpace'
datawarehouse_path = f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}'

datawarehouse_engine = create_engine(datawarehouse_path)


In [3]:
# Extracting data from Source

with source_engine.begin():
    try:
        query = ('SELECT distinct Patient_Primary_Insurance_Name FROM AceExtraction.Medrite_input_table;')
        insurance_names_df = pd.read_sql(query, source_engine)
        insurance_names = list(insurance_names_df['Patient_Primary_Insurance_Name'])
    except Exception as e:
        print('Error: ',e)

#insurance_names_df
#insurance_names

In [4]:
#creating empty dataframe

insurance_firms_dim_df = pd.DataFrame(columns=['ins_firm_id', 'ins_firm_code', 'ins_firm_name', 'ins_firm_created_date'])

#insurance_firms_dim_df

In [5]:
#Loading the data into dataframe

insurance_firms_dim_df['ins_firm_name'] = insurance_names

In [6]:
#Defining the data types

data_types = {'ins_firm_id': types.Integer(),
              'ins_firm_code':  types.String(255),
              'ins_firm_name':  types.String(255),
              'ins_firm_created_date': types.Date()}

In [7]:
try:
 insurance_firms_dim_df.to_sql('insurance_firms_dim',staging_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into Staging_DB')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into Staging_DB


##ETL from staging to DataWarehouse

In [8]:
#Extracting data from staging DB

with staging_engine.begin():
 try:
     query = ("select distinct ins_firm_name from staging_source.insurance_firms_dim;")
     staging_insurance_firms_df = pd.read_sql_query(query, staging_engine)
     staging_insurance_firms_df.dropna(subset=['ins_firm_name'], inplace=True)
     staging_insurance_firms = list(staging_insurance_firms_df['ins_firm_name'])
 except Exception as e:
     print("An error occurred:", e)

In [9]:
#creating empty dataframe

insurance_firms_dim_df = pd.DataFrame(columns=['ins_firm_id', 'ins_firm_code', 'ins_firm_name', 'ins_firm_created_date'])

#insurance_firms_dim_df

In [10]:
#Determining the current maximum ID in the DataFrame, 
current_max_id = insurance_firms_dim_df['ins_firm_id'].max() if not insurance_firms_dim_df.empty else 0
# Using pandas Series to generate IDs
new_ids = pd.Series(range(current_max_id + 1, current_max_id + 1 + len(staging_insurance_firms)))


In [11]:
#TRANSFORMATION 

new_records = pd.DataFrame({
 'ins_firm_id': new_ids,
 'ins_firm_code':[category[:3].upper() + str('_') + str(new_id) for category, new_id in zip(staging_insurance_firms, new_ids)],
 'ins_firm_name': staging_insurance_firms,
 'ins_firm_created_date': [datetime.now().strftime('%Y-%m-%d')] * len(staging_insurance_firms)
})

In [12]:
# Appending the new records to the original DataFrame

insurance_firms_dim_df = pd.concat([insurance_firms_dim_df, new_records], ignore_index=True)

In [13]:
#Defining the data types

data_types = {'ins_firm_id': types.Integer(),
              'ins_firm_code':  types.String(255),
              'ins_firm_name':  types.String(255),
              'ins_firm_created_date': types.Date()}

In [14]:
#Loading the data into Datawarehouse

try:
 insurance_firms_dim_df.to_sql('insurance_firms_dim',datawarehouse_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into DataWarehouse')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into DataWarehouse
